In [70]:
#1. 파일 불러오기
#2. 각 줄을 하나의 document로
#3. 만약 길이 길면 호단위로 자르기

In [71]:
import json
import re
import os
from pathlib import Path
import glob

In [72]:
try:
  from langchain_core.documents import Document
except ImportError:
  !pip install langchain_core
  from langchain_core.documents import Document

try:
  from langchain_text_splitters import RecursiveCharacterTextSplitter
except ImportError:
  !pip install langchain_text_splitters
  from langchain_text_splitters import RecursiveCharacterTextSplitter

In [73]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1400,
    chunk_overlap=200,
    separators = ["다.", "다만", ".", ",", " ", ""]
)

In [74]:
documents = []

# path = "./drive/MyDrive/rag/preprocessing_rag/법인세법 시행규칙.jsonl"
for file_path in glob.glob("/content/drive/MyDrive/rag/preprocessing_rag/*.jsonl"):
  with open(file_path, 'r', encoding="utf-8") as f:
    for no, line in enumerate(f, 1):
      line = line.strip()
      if not line:
        continue
      data = json.loads(line)
      content = data.get('chunk_text')

      metadata = dict(data)
      metadata.pop("chunk_text")

      metadata['조문_index'] = metadata.pop("chunk_index")
      metadata['file_line_no'] = no

      if len(content) > 1400:
        pieces = text_splitter.split_text(content)
        for i, p in enumerate(pieces, 1):
          metadata = metadata.copy()
          metadata['piece_index'] = i
          metadata['piece_count'] = len(pieces)
          documents.append(Document(page_content=p, metadata=metadata))
      else:
        metadata['piece_index'] = 1
        metadata['piece_count'] = 1
        documents.append(Document(page_content=content, metadata=metadata))


for i, document in enumerate(documents[-10:], 1):
  print(f"청크 {i}번")
  print(f"청크 내용 {document.page_content}")
  print('-'*30)
  print(f"메타데이터 {document.metadata}")
  print("\n")

print(len(documents))

청크 1번
청크 내용 제225조의3 제225조의3 삭제 <2021.2.17>
------------------------------
메타데이터 {'law_name': '소득세법 시행령', 'law_id': '003956', '공포일자': '20251230', '시행일자': '20260102', '조문번호': '225', '조문가지번호': '3', '조문제목': '', 'level': '조문내용', '항번호': None, '호번호': None, '목번호': None, 'source': '법령정보센터', '조문_index': 1, 'file_line_no': 3824, 'piece_index': 1, 'piece_count': 1}


청크 2번
청크 내용 제226조(표본조사 등)
------------------------------
메타데이터 {'law_name': '소득세법 시행령', 'law_id': '003956', '공포일자': '20251230', '시행일자': '20260102', '조문번호': '226', '조문가지번호': None, '조문제목': '표본조사 등', 'level': '조문내용', '항번호': None, '호번호': None, '목번호': None, 'source': '법령정보센터', '조문_index': 1, 'file_line_no': 3825, 'piece_index': 1, 'piece_count': 1}


청크 3번
청크 내용 ① 법 제175조제1항에서 "대통령령으로 정하는 자"란 기부금세액공제 대상금액 또는 필요경비 산입금액이 100만원 이상인 거주자 또는 법 제121조제2항 및 제5항에 따른 비거주자를 말한다. <개정 2013.2.15, 2014.2.21>
------------------------------
메타데이터 {'law_name': '소득세법 시행령', 'law_id': '003956', '공포일자': '20251230', '시행일자': '20260102', '조문번호': '226', '조문가지번호': No

In [75]:
try:
  from langchain_openai import OpenAIEmbeddings
except ImportError:
  !pip install langchain_openai
  from langchain_openai import OpenAIEmbeddings

try:
  import chromadb
except ImportError:
  !pip install chromadb
  import chromadb

try:
  from openai import OpenAI
except ImportError:
  !pip install openai
  from openai import OpenAI

In [77]:
db_path = "/content/drive/MyDrive/rag/chromadb"

client = chromadb.PersistentClient(path=db_path)
collection = client.get_or_create_collection(name = "law")

def none_case(value, name):
  return str(value).strip() if value not in [None, ""] else f"NO_{name}"

def doc_id(metadata):
  law_id = metadata.get("law_id")
  line_no = metadata.get("file_line_no")
  조문_index = metadata.get("조문_index")
  piece_index = metadata.get("piece_index")
  # line_no, 조문_index, piece_index 모두 1부터 시작

  조문번호 = none_case(metadata.get("조문번호"), "조문번호")
  조문가지번호 = none_case(metadata.get("조문가지번호"), "조문가지번호")
  항번호 = none_case(metadata.get("항번호"), "항번호")
  호번호 = none_case(metadata.get("호번호"), "호번호")
  목번호 = none_case(metadata.get("목번호"), "목번호")

  key = f"{조문번호}_{조문가지번호}_{항번호}_{호번호}_{목번호}"

  return f"{law_id}:{key}:j{조문_index}:p{piece_index}:l{line_no}"

documents_content = [doc.page_content for doc in documents]
documents_metadatas = [doc.metadata for doc in documents]
documents_ids = [doc_id(doc.metadata) for doc in documents]


In [78]:
from google.colab import userdata

embedding_model = "text-embedding-3-small"

openai_ = OpenAI(api_key = userdata.get('OPENAI_API_KEY'))

def embedding(doc):
  embeddings = openai_.embeddings.create(model=embedding_model, input=doc)
  return [emb.embedding for emb in embeddings.data]


In [79]:
def batch_upsert(documents, metadatas, ids, collection = collection, batch_size=500):

  total = 0
  for i in range(0, len(documents), batch_size):
    batch_docs = documents[i:i+batch_size]
    batch_metadatas = metadatas[i:i+batch_size]
    batch_ids = ids[i:i+batch_size]

    batch_embeddings = embedding(batch_docs)
    collection.upsert(documents = batch_docs, metadatas = batch_metadatas, ids=batch_ids, embeddings=batch_embeddings)
    total += len(batch_docs)

  return total


In [80]:
batch_upsert(documents_content, documents_metadatas, documents_ids)

14850

In [81]:
print(collection.count())

res = collection.get(ids=documents_ids[-1])
print(res.keys())

14850
dict_keys(['ids', 'embeddings', 'documents', 'uris', 'included', 'data', 'metadatas'])


In [82]:
print(len(documents_ids), len(set(documents_ids)))

14850 14850


In [83]:
print(res["documents"][-1][:200])
print(res["metadatas"][-1])


② 납세지 관할 세무서장은 위반행위의 정도, 위반 횟수, 위반행위의 동기와 그 결과 등을 고려해 별표 5에 따른 과태료 금액의 2분의 1의 범위에서 그 금액을 줄이거나 늘릴 수 있다. 다만, 과태료 금액을 늘리는 경우에는 법 제177조에 따른 과태료 금액의 상한을 넘을 수 없다. <개정 2021.2.17>
{'조문제목': '과태료의 부과기준', 'piece_index': 1, 'source': '법령정보센터', '항번호': '②', '조문_index': 3, 'level': '항', 'file_line_no': 3833, 'law_id': '003956', '조문번호': '228', '시행일자': '20260102', '공포일자': '20251230', 'law_name': '소득세법 시행령', 'piece_count': 1}


In [84]:
print(res["ids"][-1])
print(res["metadatas"][-1])

003956:228_NO_조문가지번호_②_NO_호번호_NO_목번호:j3:p1:l3833
{'조문제목': '과태료의 부과기준', 'piece_index': 1, 'source': '법령정보센터', '항번호': '②', '조문_index': 3, 'level': '항', 'file_line_no': 3833, 'law_id': '003956', '조문번호': '228', '시행일자': '20260102', '공포일자': '20251230', 'law_name': '소득세법 시행령', 'piece_count': 1}


In [85]:
client.list_collections()

[Collection(name=law)]

In [86]:
collection.count()

14850